<a href="https://colab.research.google.com/github/SpringBoardMentro0743/Ridewise_Project_2025/blob/MaahiPal/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask pyngrok joblib

In [ ]:
!ngrok config add-authtoken 33YdGaEaF7ShweB8fbjI9Vjc13P_4kXxnz9wFneNJd9KggAmJ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hour.csv to hour.csv
Saving pipeline.pkl to pipeline.pkl


In [ ]:
import os
os.makedirs("templates", exist_ok=True)
os.makedirs("static", exist_ok=True)

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Bike Rental Prediction</title>
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Segoe+UI:wght@300;400;600;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="main-layout">
        <div class="left-panel">
            <h1>Bike Rental Prediction</h1>
            <form id="prediction-form">
                <div class="form-grid">
                    <div class="form-group">
                        <label for="season">Season</label>
                        <select id="season" name="season">
                            <option value="1">Spring</option>
                            <option value="2" selected>Summer</option>
                            <option value="3">Fall</option>
                            <option value="4">Winter</option>
                        </select>
                    </div>
                    <div class="form-group">
                        <label for="yr">Year</label>
                        <select id="yr" name="yr">
                            <option value="0">2011</option>
                            <option value="1" selected>2012</option>
                        </select>
                    </div>
                    <div class="form-group">
                        <label for="mnth">Month</label>
                        <select id="mnth" name="mnth">
                            <option value="1">January</option><option value="2">February</option><option value="3">March</option><option value="4">April</option><option value="5">May</option><option value="6" selected>June</option><option value="7">July</option><option value="8">August</option><option value="9">September</option><option value="10">October</option><option value="11">November</option><option value="12">December</option>
                        </select>
                    </div>
                     <div class="form-group">
                        <label for="weekday">Day of the Week</label>
                        <select id="weekday" name="weekday">
                            <option value="0">Sunday</option><option value="1">Monday</option><option value="2">Tuesday</option><option value="3">Wednesday</option><option value="4" selected>Thursday</option><option value="5">Friday</option><option value="6">Saturday</option>
                        </select>
                    </div>
                    <div class="hour-slider-group">
                        <label for="hr">Hour of the Day</label>
                        <div class="slider-wrapper">
                            <input type="range" id="hr" name="hr" min="0" max="23" value="17">
                            <span id="hour-value">17</span>
                        </div>
                    </div>
                    <div class="form-group">
                        <label for="holiday">Holiday (0:No, 1:Yes)</label>
                        <input type="number" id="holiday" name="holiday" value="0" required>
                    </div>
                    <div class="form-group">
                        <label for="workingday">Working Day (0:No, 1:Yes)</label>
                        <input type="number" id="workingday" name="workingday" value="1" required>
                    </div>
                    <div class="form-group">
                        <label for="weathersit">Weather Situation (1-4)</label>
                        <input type="number" id="weathersit" name="weathersit" value="1" required>
                    </div>
                     <div class="form-group">
                        <label for="temp">Temperature (Normalized)</label>
                        <input type="number" id="temp" name="temp" step="0.01" value="0.5" required>
                    </div>
                    <div class="form-group">
                        <label for="atemp">"Feels Like" Temp (Normalized)</label>
                        <input type="number" id="atemp" name="atemp" step="0.01" value="0.48" required>
                    </div>
                    <div class="form-group">
                        <label for="hum">Humidity (Normalized)</label>
                        <input type="number" id="hum" name="hum" step="0.01" value="0.55" required>
                    </div>
                    <div class="form-group">
                        <label for="windspeed">Windspeed (Normalized)</label>
                        <input type="number" id="windspeed" name="windspeed" step="0.01" value="0.2" required>
                    </div>
                </div>
                <button type="submit">Predict</button>
            </form>
            <div id="result">
            </div>
        </div>
        <div class="right-panel">
            <img src="https://citybikez.com/images/bike-poster.png" alt="Bike Illustration" class="bike-image">
        </div>
    </div>
    <script src="{{ url_for('static', filename='script.js') }}"></script>
</body>
</html>

Writing templates/index.html


In [ ]:
%%writefile static/style.css
:root {
    /*Color Palette*/
    --primary-color: #8A2BE2;
    --primary-hover: #9932CC;
    --text-color: #333;
    --label-color: #555;
    --container-bg: #FFFFFF;
    --body-bg-start: #E6E6FA;
    --body-bg-end: #F0F0F0;
    --input-bg: #F8F8FF;
    --border-color: #D3D3D3;
    --success-color: #4CAF50;
    --right-panel-bg: #C3B1E1;
}
body {
    font-family: 'Segoe UI', sans-serif;
    background: linear-gradient(135deg, var(--body-bg-start) 0%, var(--body-bg-end) 100%);
    display: flex;
    justify-content: center;
    align-items: center;
    min-height: 100vh;
    margin: 0;
    color: var(--text-color);
    overflow: hidden;
}
.main-layout {
    display: flex;
    width: 100%;
    max-width: 1200px;
    height: 90vh;
    background-color: var(--container-bg);
    border-radius: 12px;
    box-shadow: 0 8px 40px rgba(0, 0, 0, 0.1);
    border: 1px solid rgba(0,0,0,0.05);
    overflow: hidden;
}
.left-panel {
    flex: 1;
    padding: 2rem 3rem;
    overflow-y: auto;
}
.right-panel {
    flex: 1;
    background-color: var(--right-panel-bg);
    display: flex;
    justify-content: center;
    align-items: center;
    overflow: hidden;
    border-left: 1px solid rgba(0,0,0,0.05);
}
.bike-image {
    width: 90%;
    height: auto;
    object-fit: contain;
    filter: grayscale(0.2) brightness(0.95);
}
h1 {
    text-align: center;
    color: var(--text-color);
    font-weight: 600;
    margin-bottom: 1.5rem;
}
.form-grid {
    display: grid;
    grid-template-columns: repeat(2, 1fr);
    gap: 1.2rem;
}
.form-group, .hour-slider-group {
    display: flex;
    flex-direction: column;
}
.hour-slider-group {
    grid-column: 1 / -1;
}
.slider-wrapper {
    display: flex;
    align-items: center;
    gap: 1rem;
}
#hour-value {
    font-weight: bold;
    font-size: 1.1rem;
    color: var(--primary-color);
    min-width: 25px;
    text-align: center;
}
label {
    margin-bottom: 0.5rem;
    font-weight: 400;
    font-size: 0.9rem;
    color: var(--label-color);
}
input, select {
    font-family: 'Segoe UI', sans-serif;
    padding: 0.75rem;
    border: 1px solid var(--border-color);
    border-radius: 6px;
    font-size: 1rem;
    background-color: var(--input-bg);
    color: var(--text-color);
    transition: border-color 0.2s, box-shadow 0.2s;
}
select {
    -webkit-appearance: none;
    background-image: url('data:image/svg+xml;charset=US-ASCII,%3Csvg%20width%3D%2220%22%20height%3D%2220%22%20xmlns%3D%22http%3A//www.w3.org/2000/svg%22%3E%3Cpath%20d%3D%22M5%208l5%205%205-5z%22%20fill%3D%22%23555%22/%3E%3C/svg%3E');
    background-repeat: no-repeat;
    background-position: right 0.7rem top 50%;
    background-size: .65rem auto;
}
input:focus, select:focus {
    outline: none;
    border-color: var(--primary-color);
    box-shadow: 0 0 0 3px rgba(138, 43, 226, 0.15);
}
input[type="range"] {
    -webkit-appearance: none; appearance: none;
    width: 100%; height: 8px;
    background: #e0e0e0;
    border-radius: 5px; outline: none; padding: 0;
}
input[type="range"]::-webkit-slider-thumb,
input[type="range"]::-moz-range-thumb {
    -webkit-appearance: none;
    width: 20px; height: 20px;
    background: var(--primary-color);
    border-radius: 50%; cursor: pointer; border: none;
    transition: background-color 0.2s;
}
input[type="range"]::-webkit-slider-thumb:hover,
input[type="range"]::-moz-range-thumb:hover {
    background-color: var(--primary-hover);
}
button {
    grid-column: 1 / -1;
    padding: 0.8rem;
    background-color: var(--primary-color);
    color: var(--container-bg);
    border: none;
    border-radius: 6px;
    font-family: 'Segoe UI', sans-serif;
    font-size: 1.1rem;
    font-weight: bold;
    cursor: pointer;
    margin-top: 1rem;
    transition: background-color 0.2s;
}
button:hover {
    background-color: var(--primary-hover);
}
#result {
    margin-top: 2rem;
    text-align: center;
    font-size: 1.5rem;
    font-weight: bold;
    color: var(--success-color);
}
@media (max-width: 850px) {
    .main-layout { flex-direction: column; height: auto; max-height: 100vh; margin: 1rem; }
    .right-panel { min-height: 300px; order: -1; border-left: none; border-top: 1px solid rgba(0,0,0,0.05); }
    .left-panel { padding: 1.5rem 2rem; }
}
@media (max-width: 500px) {
    .form-grid, .hour-slider-group { grid-template-columns: 1fr; grid-column: 1 / -1; }
}

Writing static/style.css


In [ ]:
%%writefile static/script.js
const hourSlider = document.getElementById('hr');
const hourValueSpan = document.getElementById('hour-value');

hourSlider.addEventListener('input', () => {
    hourValueSpan.textContent = hourSlider.value;
});

document.getElementById('prediction-form').addEventListener('submit', async function(event) {
    event.preventDefault();

    // Collect ALL features expected by the ML model
    const data = {
        season: parseInt(document.getElementById('season').value),
        mnth: parseInt(document.getElementById('mnth').value),
        weekday: parseInt(document.getElementById('weekday').value),
        hr: parseInt(document.getElementById('hr').value),
        holiday: parseInt(document.getElementById('holiday').value),
        workingday: parseInt(document.getElementById('workingday').value),
        weathersit: parseInt(document.getElementById('weathersit').value),
        temp: parseFloat(document.getElementById('temp').value),
        atemp: parseFloat(document.getElementById('atemp').value),
        hum: parseFloat(document.getElementById('hum').value),
        windspeed: parseFloat(document.getElementById('windspeed').value)
    };

    try {
        const response = await fetch("/predict", {
            method: "POST",
            headers: { "Content-Type": "application/json" },
            body: JSON.stringify(data)
        });

        const result = await response.json();

        const finalPrediction = Math.max(0, Math.round(result.prediction));

        document.getElementById('result').textContent = `Predicted Rentals: ${finalPrediction}`;
    } catch (error) {
        document.getElementById('result').textContent = `Error: ${error}`;
    }
});

Writing static/script.js


In [ ]:
import pandas as pd
rental_data = pd.read_csv("hour.csv")

In [ ]:
from flask import Flask, request, jsonify, render_template
from pyngrok import ngrok
import threading
import numpy as np
import joblib

app = Flask(__name__)

def create_features(df):
    df = df.copy()

    df["hr_sin"] = np.sin(2 * np.pi * df["hr"] / 24.0)
    df["hr_cos"] = np.cos(2 * np.pi * df["hr"] / 24.0)
    df["mnth_sin"] = np.sin(2 * np.pi * df["mnth"] / 12.0)
    df["mnth_cos"] = np.cos(2 * np.pi * df["mnth"] / 12.0)
    df["weekday_sin"] = np.sin(2 * np.pi * df["weekday"] / 7.0)
    df["weekday_cos"] = np.cos(2 * np.pi * df["weekday"] / 7.0)
    df["season_sin"] = np.sin(2 * np.pi * df["season"] / 4.0)
    df["season_cos"] = np.cos(2 * np.pi * df["season"] / 4.0)

    df["is_weekend"] = df["weekday"].apply(lambda x: 1 if x in [0, 6] else 0)
    df["is_morning_peak"] = df["hr"].apply(lambda x: 1 if x in [7, 8, 9] else 0)
    df["is_evening_peak"] = df["hr"].apply(lambda x: 1 if x in [17, 18, 19] else 0)

    last_24_cnt = rental_data["cnt"].iloc[-24:]
    df["lag_1"] = last_24_cnt.iloc[-1]
    df["lag_24"] = last_24_cnt.iloc[0]
    df["rolling_24h"] = last_24_cnt.mean()

    return df

pipeline = joblib.load("pipeline.pkl")

@app.route("/")
def home():
    """Renders the home page."""
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
    try:
        data = request.json
        df = pd.DataFrame([data])
        df = df.astype(float)
        prediction = pipeline.predict(df)[0]
        return jsonify({"prediction": max(0, float(prediction))})
    except Exception as e:
        return jsonify({"error": str(e)})


def run_flask():
    app.run(port=5050, debug=False, use_reloader=False)

thread = threading.Thread(target=run_flask)
thread.daemon = True
thread.start()

try:
    public_url = ngrok.connect(5050)
    print("Your app is live!")
    print("Public URL:", public_url)
except Exception as e:
    print(f"Error starting ngrok: {e}")

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5050
INFO:werkzeug:Press CTRL+C to quit


Your app is live!
Public URL: NgrokTunnel: "https://pentavalent-davin-nonsubliminally.ngrok-free.dev" -> "http://localhost:5050"
